<h2> Homework 6 </h2>

Выполняла работу Куканова Абина, БКЛ-212

In [97]:
#скачиваем все нужные библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

In [98]:
#открываем файлы
df1 = pd.read_csv('train.csv', delimiter=',')
df2 = pd.read_csv('test.csv', delimiter=',')

In [153]:
def pre_processing(text):
    text = re.sub(r'[^\w\s]','',text)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

In [154]:
df1['text'] = df1.text.apply(lambda x: pre_processing(x))
data_train, data_test, y_train, y_test = train_test_split(df1['text'], df1['answer'], test_size=0.2, random_state=0)

In [156]:
stops = stopwords.words("english")
tfidf = TfidfVectorizer(stop_words=stops, binary=True, max_features=10000)
tfidf.fit(data_train)
X_train = tfidf.transform(data_train)
X_test = tfidf.transform(data_test)

Логистическая регрессия

In [161]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.8754

Деревья решений

In [163]:
model = DecisionTreeClassifier(max_depth=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.691

Random Forest(Случайный Лес)

In [165]:
model = RandomForestClassifier(
    n_estimators=20,
    max_depth=5,
    max_samples=0.5,
    max_features=1000,
    random_state=0,
)

model.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, max_features=1000, max_samples=0.5,
                       n_estimators=20, random_state=0)

In [166]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.7524

Машина опорных векторов

In [168]:
svm = SVC(kernel='linear', C=1.0, random_state=1)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
accuracy_score(y_test, y_pred)

0.8728

Класс KNeighborsClassifier в Scikit-learn (Метод ближайших соседей)

In [170]:
knn = KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
accuracy_score(y_test, y_pred)

0.8728

Найвный байесовский метод

In [172]:
model = MultinomialNB().fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.8564

Среди всех шести моделей, модель Логистической регрессий показала лучший результат, поэтому сохраняем результат этой модели в файл(0.8754).

In [173]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

X_test_result = tfidf.transform(df2['text'])
y_pred = model.predict(X_test_result)
result = df2[["id"]].copy()
result["answer"] = y_pred
result.to_csv("result.csv", index=False)

В общем я использовала 6 моделей: логистическую регрессию, деревье решений, машину опорных векторов, класс KNeighborsClassifier в Scikit-learn(Метод ближайших соседей), random forest(случайный лес), найвный байесовский метод. Получила следующие результаты:

<ol>
 <li>Логистичесская регрессия - 0.8754 </li>
 <li>Деревье решений - 0.691</li>
 <li>Случайный лес - 0.7524</li>
 <li>Машина опорных векторов - 0.8728</li>
 <li>Класс KNeighborsClassifier в Scikit-learn - 0.8728</li>
 <li>Найвный байесовский метод - 0.8564</li>
</ol>

Модель машины опорных векторов загружался довольно дольше остальных, а другие модели работали очень быстро. Результаты моделей машины опорных векторов и метода ближайших соседей одинаковые(0.8728). Модель деревья решений показала самый низкий результат среди всех моделей(0.691).
Думаю, что по логике все модели кроме машины опорных векторов хорошие, так как работают очень быстро, и у трех из них качество было больше 0.85. А со стороны качества, то скорее всего, модель машины опорных векторов показалась лучше, так как та проводит более долгую работу.